In [1]:
import minsearch
import json

# Read data

In [2]:
with open('documents.json', 'r') as f_in:
    raw_doc = json.load(f_in)

In [3]:
documents = []
for course_dict in raw_doc:
    course = course_dict['course']
    for doc in course_dict['documents']:
        doc['course'] = course
        documents.append(doc)

# Create minsearch index

In [4]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [5]:
index.fit(documents)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
boost = {'question': 3.0, 'section': 0.5}
index.search(
    query=q,
    boost_dict=boost,
    num_results=5
)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the cour

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result = index.search(
        query=q,
        boost_dict=boost,
        num_results=5
    )
    return result

# Use LLM for generation answer

### OpenAI

In [9]:
from openai import OpenAI

In [10]:
clientO = OpenAI()

In [11]:
response = clientO.chat.completions.create(
    model='gpt-4o',
    messages = [
        {
            'role': 'user',
            'content': q
        }
    ]
)
print(response.choices[0].message.content)

Yes, many courses allow for late enrollment, especially if they are offered online or have flexible start dates. However, policies on late enrollment can vary depending on the institution or platform offering the course. Here are some steps you can take:

1. **Check the Course Website:** Look for information regarding late enrollment or contact details.
2. **Contact the Instructor or Administration:** Reach out to the instructor or program administrator to inquire about the possibility of joining late. Explain your situation and express your interest.
3. **Review Course Requirements:** Make sure you understand what you might have missed and determine if you can catch up on your own or with some assistance.
4. **Consider Alternative Options:** If late enrollment isn't possible, ask if there are other start dates, similar courses, or any recorded sessions that you can attend later.

Be sure to act quickly, as the sooner you reach out, the better your chances of being able to join the cou

### Anthropic

In [13]:
from anthropic import Anthropic

In [14]:
clientA = Anthropic()

In [15]:
response = clientA.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens = 500,
    messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": q
            }
          ]
        }
  ]
)
print(response.content[0].text)

The answer to whether you can still enroll in a course that has already started depends on the specific policies and procedures of the educational institution offering the course. Here are a few key things to consider:

1. Enrollment deadlines: Most institutions have strict enrollment deadlines, often within the first week or two of the course starting. If you try to enroll after this deadline has passed, you may not be able to do so.

2. Late enrollment: Some institutions may allow late enrollment, but this is usually subject to certain conditions. For example, you may need to obtain special permission from the instructor or department, or there may be additional fees associated with late enrollment.

3. Missed content: If you do manage to enroll after the course has already started, you will likely have missed some of the course material and activities. It's important to consider whether you can catch up on the missed content and meet the course requirements.

4. Attendance and parti

# Put together

In [16]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [17]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [18]:
def llm_openai(prompt):
    response = clientO.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [19]:
def llm_anth(prompt):
    response = clientA.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens = 500,
        messages=[
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": prompt
                }
              ]
            }
      ]
    )
    return response.content[0].text

In [20]:
def rag_openai(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_openai(prompt)
    return answer

def rag_anth(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_anth(prompt)
    return answer

In [21]:
q = "how do I run kafka?"

In [22]:
rag_openai(q)

"To run Kafka producer, consumer, kstreams, etc., in the terminal, follow these steps from the project directory:\n\n1. If you're working with Java, use the following command:\n    ```\n    java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n    ```\n\n2. If you're working with Python and encounter issues:\n    - Create a virtual environment and install the necessary packages:\n        ```\n        python -m venv env\n        source env/bin/activate\n        pip install -r ../requirements.txt\n        ```\n    - Activate the virtual environment each time you need it:\n        ```\n        source env/bin/activate\n        ```\n    - Deactivate the environment when done:\n        ```\n        deactivate\n        ```\n    Note that for Windows, the command to activate the virtual environment is slightly different:\n    ```\n    env\\Scripts\\activate\n    ```\n\nEnsure that Docker images are up and running before creating the virtual environmen

In [43]:
rag_anth(q)

'Based on the context provided in the FAQ database, the answer to the question "how do I run kafka?" is:\n\nTo run Kafka, you can follow these steps:\n\n1. **Java Kafka**:\n   - In the project directory, run the following command to run the Java Kafka producer, consumer, KStreams, etc.:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Python Kafka**:\n   - Create a virtual environment and install the necessary packages:\n     ```\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - Activate the virtual environment before running the Python files:\n     ```\n     source env/bin/activate\n     ```\n   - To deactivate the virtual environment:\n     ```\n     deactivate\n     ```\n\nNote that the virtual environment should be created only to run the Python file, and Docker images should be up and running first.'

# Use Elastic search

Run elastic search image
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

In [1]:
from elasticsearch import Elasticsearch

In [24]:
es_client = Elasticsearch('http://localhost:9200')

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
from tqdm.auto import tqdm

/home/roman/.local/share/virtualenvs/llm-zoomcamp-6WKyO3I_/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████| 948/948 [00:08<00:00, 114.92it/s]


In [29]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [30]:
elastic_search(q)

[{'text': "Answer: To run the provided code, ensure that the 'dlt[duckdb]' package is installed. You can do this by executing the provided installation command: !pip install dlt[duckdb]. If you’re doing it locally, be sure to also have duckdb pip installed (even before the duckdb package is loaded).",
  'section': 'Workshop 1 - dlthub',
  'question': 'How do I install the necessary dependencies to run the code?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'After you create a GitHub account, you should clone the course repo to your local machine using the process outlined in this video: Git for Everybody: How to Clone a Repository from GitHub\nHaving this local repositor

In [31]:
def rag_openai_es(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm_openai(prompt)
    return answer

In [32]:
rag_openai_es(q)

'To run Kafka, follow the instructions provided for running Java Kafka producer/consumer/KStreams/etc in the terminal from the course content:\n\nIn the project directory, execute the following command:\n```bash\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file. This command will start the desired Kafka component (producer, in this example) specified in the Java class file `JsonProducer.java`.'